In [1]:
import pandas as pd
import numpy as np

In [2]:
X_train = catalog.load("X_train")
X_test = catalog.load("X_test")
y_train = catalog.load("y_train")
y_test = catalog.load("y_test")
CT = catalog.load("CT")

[11/26/23 14:29:11] INFO     Loading data from 'X_train' (CSVDataSet)...                        ]8;id=81860;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=604504;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'X_test' (CSVDataSet)...                         ]8;id=467935;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=774057;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'y_train' (CSVDataSet)...                        ]8;id=613750;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=980129;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'y_test' (CSVDataSet)...                         ]8;id=236623;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=780377;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'CT' (PickleDataSet)...                          ]8;id=338711;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=839785;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [3]:
from sklearn.cluster import KMeans

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline

from sklearn.model_selection import KFold
import numpy as np

def plot_elbow_kfold(data, CT, max_clusters=10, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True)
    inertias = []
    data = CT.fit_transform(data)
    for k in range(1, max_clusters + 1):
        fold_inertias = []
        print(f"running k={k}")
        for train_index, _ in kf.split(data):
            kmeans = KMeans(n_clusters=k)
            kmeans.fit(data[train_index])
            fold_inertias.append(kmeans.inertia_)

        inertias.append(np.mean(fold_inertias))

    # Plotting the elbow plot
    sns.set(style="whitegrid")
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, max_clusters + 1), inertias, marker='o')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Average Within-Cluster Sum of Squares (Inertia)')
    plt.title('Elbow Method using K-Fold Cross Validation')
    plt.xticks(np.arange(1, max_clusters + 1))
    plt.show()

In [5]:
X_train

,product,channel,age_bucket,operating_system,days_to_churn,product_x,product_y,is_xs
0,product_a,channel_b,45-49,Android,730.0,0.0,0.0,0
1,product_a,channel_b,25-29,iOS,449.0,0.0,0.0,0
2,product_a,channel_b,25-29,Android,226.0,0.0,0.0,0
3,product_a,channel_a,30-34,iOS,2020.0,0.0,0.0,0
4,product_a,channel_b,25-29,iOS,1461.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...
82093,product_a,channel_b,25-29,iOS,327.0,0.0,0.0,0
82094,product_a,channel_b,18-24,Android,791.0,0.0,0.0,0
82095,product_a,channel_a,25-29,iOS,507.0,0.0,0.0,0
82096,product_a,channel_b,35-39,iOS,1339.0,0.0,0.0,0


In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
# plot_elbow_kfold(X_train, CT, max_clusters=10, n_splits=5)

## Procceeding with k=6

In [8]:
k=6
n_splits=5
kf = KFold(n_splits=n_splits, shuffle=True)
kmeans = KMeans(n_clusters=k)
pipe = Pipeline([("CT", CT), ("model", kmeans)])
pipe.fit(X_train)


Pipeline(steps=[('CT',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['product', 'channel',
                                                   'age_bucket',
                                                   'operating_system']),
                                                 ('numerical', StandardScaler(),
                                                  ['days_to_churn', 'product_x',
                                                   'product_y'])])),
                ('model', KMeans(n_clusters=6))])

In [9]:
len(np.unique(pipe["model"].labels_))

6

In [10]:
centroids = pipe["model"].cluster_centers_

In [11]:
centroids


array([[ 8.75553440e-01,  2.27088788e-02,  1.01737682e-01,
         1.00000000e+00,  1.65423231e-14,  1.12116163e-01,
         1.89716734e-01,  1.98286122e-01,  1.37253035e-01,
         9.58343252e-02,  6.89835753e-02,  6.32706498e-02,
         1.33730064e-01,  8.09331112e-04,  4.96358010e-01,
         4.34848846e-01,  6.87931445e-02, -6.89951553e-01,
        -3.08477369e-02, -9.66290973e-02],
       [ 6.79874938e-01,  8.69507981e-02,  2.33174264e-01,
        -2.39808173e-14,  1.00000000e+00,  1.32236301e-01,
         2.58877736e-01,  2.28632549e-01,  1.42833635e-01,
         8.82672371e-02,  5.28879381e-02,  3.78147112e-02,
         5.77916735e-02,  6.58219516e-04,  4.60095442e-01,
         4.62300477e-01,  7.76040810e-02, -6.06506292e-01,
        -3.08477369e-02, -9.66290973e-02],
       [ 9.28670186e-01,  1.13750628e-02,  5.99547511e-02,
         1.00000000e+00,  1.88737914e-15,  6.05832076e-02,
         1.37820513e-01,  1.60256410e-01,  1.09665661e-01,
         1.01181498e-01,  8.

In [31]:
def top_cluster_features(data, _pipe, top_n=3):  
    # Assign cluster labels to each data point
    labels = _pipe["model"].labels_
    k = len(np.unique(labels))
    data['Cluster'] = labels
    data = _pipe["CT"].transform(data)

    # Calculate centroids for each cluster
    centroids = _pipe["model"].cluster_centers_
    
    # Get feature importances for each cluster
    cluster_features = {}
    for cluster in range(k):
        centroid = centroids[cluster]
        abs_centroid = np.abs(centroid)  # Take absolute values for importance
        
        # Sort indices by importance and get top_n
        top_indices = np.argsort(abs_centroid)[::-1][:top_n]
        
        # Get feature names and importance values
        feature_names = _pipe[0].get_feature_names_out()[top_indices]
        feature_importance = centroid[top_indices]
        
        cluster_features[cluster] = pd.DataFrame({'Features': feature_names, 'Importance': feature_importance})
    
    return cluster_features

,product,channel,age_bucket,operating_system,days_to_churn,product_x,product_y,is_xs
0,product_a,channel_a,55+,Android,79.0,0.0,0.0,0


0

In [37]:
top_cluster_features(X_train, pipe)[pipe.predict(X_test.iloc[:1])[0]].to_json(orient="values")

'[["categorical__channel_channel_a",1.0],["categorical__product_product_a",0.8755534397],["numerical__days_to_churn",-0.6899515527]]'

In [ ]:
X_train.drop("product_x")